Prinipals of Urban Infomatics
Homework 5
Assignment II
By: Max Feinglass

##GENDER INCOME GAP

In [ ]:
import pandas as pd
import sys
import os
import numpy as np
import pylab as pl
import statsmodels.formula.api as smf
import statsmodels.api as sm
import json
import os
import scipy.stats as stats


%pylab inline

The Census data on income comes from a series of files posted for this class.  They can be found at  http://cosmo.nyu.edu/~fb55/UI_CUSP_2015/data.  Here I import it using panda's "read excel" function.  I excluded the 'all' racial categories, as I thought it would be redundant and skew the results of the further analysis by "double counting" certain relationships between sex, race, and pay.

First, I created a dictionary of all data pertaining to men and women respectively.  Then I populated each dictionary with the data for men and the data from women derived from the census.  

In [ ]:
url="http://cosmo.nyu.edu/~fb55/UI_CUSP_2015/data/"
#split your data by male and female, for example you can load the data into 2 disctionary. other variables may be relevant: for example race.
#the names are not informative, and i am not giving you any meta data to find out what is in each file. 
# 08_2 files are male, 08_3 are female, and I am leaving in the lines of code that read the male files into races

allmales={}
allfemales={}
#allmales['all']=pd.read_excel(url+'/pinc08_2_1_1.xls',header=9)#%s/pinc03_2_1_1.xls'%url, header=9)
allmales['white']=pd.read_excel(url+'/pinc08_2_1_4.xls', header=9)
allmales['black']=pd.read_excel(url+'/pinc08_2_1_6.xls', header=9)
allmales['hispanic']=pd.read_excel(url+'/pinc08_2_1_9.xls', header=9)
allmales['asian']=pd.read_excel(url+'/pinc08_2_1_8.xls', header=9)


In [ ]:
#allfemales['all']=pd.read_excel(url+'/pinc08_3_1_1.xls', header=9)
allfemales['white']=pd.read_excel(url+'/pinc08_3_1_4.xls', header=9)
allfemales['black']=pd.read_excel(url+'/pinc08_3_1_6.xls', header=9)
allfemales['hispanic']=pd.read_excel(url+'/pinc08_3_1_9.xls', header=9)
allfemales['asian']=pd.read_excel(url+'/pinc08_3_1_8.xls', header=9)

I did not include the scatter matrix here as it took about 10 minutes to run on my machine.  But rest assured, run it did.  If you would to confirm than all one has to do is remove the comment below.

In [ ]:
from pandas.tools.plotting import scatter_matrix
#scatter_matrix(allmales['all'], s=300, figsize=(16, 16), diagonal='kde')

These two loops remove the non-real numbers from our data.  First, a for loop parses each dictionary which are comprised of our different racial data, broken down by sex.  The nested loop parses each column of each race's data frame.  It checks to see if the column's non real sums are between 1 and 48.  If so, it replaces value with a 'NaN'.  If it is real, it codes that same value as a float.

In [ ]:
for k in allmales:
    nonreal  = allmales[k].applymap(np.isreal)
    for c in allmales[k].columns :
        if nonreal.sum()[c]>1 and nonreal.sum()[c]<49:
            allmales[k][c][~nonreal[c]] = np.float('NaN')
            allmales[k][c] = allmales[k][c].astype(float)
            
for k in allfemales:
    nonreal  = allmales[k].applymap(np.isreal)
    for c in allmales[k].columns :
        if nonreal.sum()[c]>1 and nonreal.sum()[c]<49:
            allmales[k][c][~nonreal[c]] = np.float('NaN')
            allmales[k][c] = allmales[k][c].astype(float)


    

Again, the scatterplot takes about 10 minutes to run.  If you wish to see so, all you have to do is uncomment below.

In [ ]:
#Scatter Matrix of all Males
from pandas.tools.plotting import scatter_matrix
#scatter_matrix(allmales['all'], s=300, figsize=(16, 16), diagonal='kde')

In [ ]:
#do the same for females
#Scatter Matrix of all Males
from pandas.tools.plotting import scatter_matrix
#scatter_matrix(allfemales['all'], s=300, figsize=(16, 16), diagonal='kde')

The next step involved pulling out the median incomes for each race, for each sex.  A for loop parsed every item in the women and men's racial dictionary and extracted the median income for that race.  It then stores that value in a new array, for each gender.

In [ ]:
femalemedianincomes = []
malemedianincomes =[]
for i in allfemales:
    femalemedianincomes.append(allfemales[i]['Median income'][1])

for i in allmales:
    malemedianincomes.append(allmales[i]['Median income'][1])
    

For plotting purposes, I created two arrays that will consistently label and plot races with a the same words and colors.

In [ ]:
races = ['white', 'black', 'hispanic', 'asian']
colors = ['blue','black','brown','pink']

I plotted each point separately using a for loop to keep the formatting simple.  What came out was a scatter plot that compared men and women, of different races, income against one another.

In [ ]:
pl.figure(figsize=(10,10))
xlabel('Median Income of Men')
ylabel('Median Income of Women')
pl.xlim([0, 50000])
pl.ylim([0, 50000])
pl.title ('Median Income of Men and Women')

for i in range(len(races)):
    pl.scatter(malemedianincomes[i], femalemedianincomes[i], alpha = 1, s=80, color = colors[i], label = races[i])

plot([0, 50000], [0, 50000], color = 'black', alpha = 1, label = "45 Degree Line")
pl.plt.legend()


One can see that the data all fall below the 45 degree line, closer to the men's axis.  This indicates that for each race, men make more than women.  However, the distance from the line, or the size of the income disparity, varies by race.

Next, I regressed male incomes, as the independent variable, against women's incomes using an ordinary least mean squares routine and plotted the resulting data.  Beware!  Any regression with just 4 data points is problematic due to low sample size.  

In [ ]:
data = pd.DataFrame({'X': malemedianincomes, 'Y': femalemedianincomes})
model = smf.ols(formula = 'Y ~ X', data = data, missing = 'drop').fit()
print "Beta0 = ", model.params[0]
print "Beta1 = ", model.params[1]

In [ ]:
races = ['white', 'black', 'hispanic', 'asian']
colors = ['blue','black','brown','pink']

pl.figure(figsize=(10,10))
xlabel('Median Income of Men')
ylabel('Median Income of Women')
pl.xlim([0, 50000])
pl.ylim([0, 50000])
pl.title ('Median Income of Men and Women')

for i in range(len(races)):
    pl.scatter(malemedianincomes[i], femalemedianincomes[i], alpha = 1, s=100, color = colors[i], label = races[i])

x_axis = np.arange(0,50000, 1000)
bestfit = (model.params[1]*x_axis)+model.params[0]
plot(x_axis, bestfit, label = "OLS Best Fit")
plot([0, 50000], [0, 50000], color = 'black', alpha = 1, label = "45 degree line")
pl.plt.legend()

Here, the OLS best fit line clearly shows that that, according to linear regression that men make more than women for salrieis above $15,000

Next, the coefficients, Beta 0 and Beta 1, were solved for analytically.

In [ ]:
cor = stats.spearmanr(malemedianincomes, femalemedianincomes)
std(malemedianincomes)/std(femalemedianincomes)
beta1=cor[0]*(std(femalemedianincomes)/std(malemedianincomes))
beta0=mean(femalemedianincomes)-(beta1*mean(malemedianincomes))
print "Beta 1 = ", beta1
print "Beta 0 = ", beta0

In [ ]:
data = pd.DataFrame({'X': malemedianincomes, 'Y': femalemedianincomes})
model = smf.ols(formula = 'Y ~ X', data = data, missing = 'drop').fit()
print model.params[0]
print model.params[1]

In [ ]:
races = ['white', 'black', 'hispanic', 'asian']
colors = ['blue','black','brown','pink']

pl.figure(figsize=(10,10))
xlabel('Median Income of Men')
ylabel('Median Income of Women')
pl.xlim([0, 50000])
pl.ylim([0, 50000])
pl.title ('Median Income of Men and Women')

for i in range(len(races)):
    pl.scatter(malemedianincomes[i], femalemedianincomes[i], alpha = 1, s=100, color = colors[i], label = races[i])

x_axis = np.arange(0,50000, 1000)
bestfit = (model.params[1]*x_axis)+model.params[0]
afit = (beta1*x_axis)+beta0
plot(x_axis, bestfit, label = 'OLS Least Squares Fit')
plot(x_axis, afit, color = 'r', label = 'Analytical Best Fit')
plot([0, 50000], [0, 50000], color = 'black', alpha = 1, label = "45 Degree Line")     
pl.plt.legend()



The analytical solution turned in a line very similar to the regressed solutions.  This is something we would expect to see.

The next lines of code refuse to run on my system.  I check with other students and our T.A, Himanshu Kumawat, to try to figure out why.  After two hours of working with the T.A, we were both unable to figure out the reason for the code not functioning.  It is unaltered from the code given in the sample workbook and should, by all accounts run.  I hope I am not punished for not being able to debug something our TA was not able to help me with.  I will interpret the results from here out as best as I can.

This code removes all NaN's from the data and 'flattens' each array into a single column.

In [ ]:
all_males = np.ma.array(np.concatenate([allmales['white']['Median income'][:].values,allmales['black']['Median income'][:].values,allmales['hispanic']['Median income'][:].values,allmales['asian']['Median income'][:].values]).flatten())

all_females = np.ma.array(np.concatenate([allfemales['white']['Median income'][:].values,allfemales['black']['Median income'][:].values,allfemales['hispanic']['Median income'][:].values,allfemales['asian']['Median income'][:].values]).flatten())

allmask = [np.isnan(all_males) + np.isnan(all_females)]

all_females.mask = allmask
all_males.mask = allmask

I would have then run a regression of men vs women with this new enlarged data set.  

In [ ]:
rib = sm.add_constant(all_males[~all_males.mask])
bigmodel = sm.OLS(all_females[~all_females.mask], rib)
fit = bigmodel.fit()
print fit.summary()
alpha0 = fit.params[0]
alpha1 = fit.aparms[1]

Then, I would have plotted the new line of fit, which contains a lot more data, alongside our original OLS and analytical models.

In [ ]:
races = ['white', 'black', 'hispanic', 'asian']
colors = ['blue','black','brown','pink']

pl.figure(figsize=(10,10))
xlabel('Median Income of Men')
ylabel('Median Income of Women')
pl.xlim([0, 50000])
pl.ylim([0, 50000])
pl.title ('Median Income of Men and Women')

for i in range(len(races)):
    pl.scatter(malemedianincomes[i], femalemedianincomes[i], alpha = 1, s=100, color = colors[i], label = races[i])

x_axis = np.arange(0,50000, 1000)
bestfit = (model.params[1]*x_axis)+model.params[0]
afit = (beta1*x_axis)+beta0
bigfit = alpha1*x_axis)+alpha0
plot(x_axis, bestfit, label = 'OLS Least Squares Fit')
plot(x_axis, afit, color = 'r', label = 'Analytical Best Fit')
plot(x_axis, bigfit, color = 'yo', label = 'Larger racial fit')
plot([0, 50000], [0, 50000], color = 'black', alpha = 1, label = "45 Degree Line")     
pl.plt.legend()

For races sampled, median incomes is less than male median income for those of the same race.  White and Asian males have the highest income and make about half again as much as the women of their same race.  Black and Hispanic men have incomes relatively similar to white and Asian women.  At the bottom of the plot, black and Hispanic women earn the lowest wages of the groups studied.  The models that only used four data points are very sensitive.  They should not be trusted, for example, to show that women earn more than men below $15,000, which i believe is lower than minimum wage.  The larger sample size associated with the last regression should probably be waited by how many people participate in each sector.  For example, if very few black women work in the farm sector, the farm sector's data point when it comes to black women's income is proportionately skewing our overall picture of black female income.

I think you did not mean "stipend" which is formally 'unearned' income.  One way to give a stipend (and there are lots of other arguments for larger or smaller stipends) would be to give women a grant that would bring their income up to the 45 degree line.  This is interesting as it shows how much more women would need to be paid to meet men.  One could calculate a given's demographics distance from the 45 degree line and, say, give a tax break to women of that demographic that is proportional to the unequalness of their pay.

In [ ]:
my_income = 1000000000
women_income = my_income*alpha1 + alpha0
print woman_income

I am assuming you meant income.  My ideal income is a billion dollars.  1,000,000,000!  A woman in my position would make????  The last statstic I heard was 0.60 cents on the dollar.  So they would make mearly 600,000,000!

In [ ]:
#is the line fit similar? can you find any high legverage points?